# WordUp

## Introduction

Use Naive Bayes to classify Reddit comments as either "up" or "down" in terms of score (net of up and down votes).  I am using the personalfinance subreddit November 2015 as input.  Initial queries to calculate percentile values and extract only extreme "up" or "down" voted comments (i.e., below 3rd and above 97th percentiles) done through Google BigQuery.

## Imports

In [ ]:
%matplotlib inline
import nltk
# import csv
import unicodecsv
import re
import pandas as pd
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ggplot import *
from mpl_toolkits.mplot3d import Axes3D
import codecs
import cStringIO
from app.helpers import WordUp

## Read data from CSV file downloaded from Google bucket

Necessary helper functions for reading Unicode CSV files. See https://docs.python.org/2/library/csv.html

I WILL TRY IT WITH UNICODECSV AND SEE HOW IT WORKS.

In [ ]:
with open('data/results-20151224-154845.csv', 'rb') as f:
    # reader = UnicodeReader(f)
    reader = unicodecsv.reader(f, encoding='utf-8')
    pflist = list(reader)

In [ ]:
f = open('data/results-20151224-154845.csv', 'rb')
reader = unicodecsv.reader(f, encoding='utf-8')
row = next(reader)
row = next(reader)

In [ ]:
row

In [ ]:
pflist[0]

delete header

In [ ]:
del pflist[0] 

map to just body and score

In [ ]:
pflist_body_score = map(lambda line: [line[0], line[11]], pflist)

In [ ]:
pflist_body_score[1001]

In [ ]:
def cleanup(body):
    body = re.sub("&gt;", ">", body) # Recode HTML codes
    body = re.sub("&lt;", "<", body)
    body = re.sub("&amp;", "&", body)
    body = re.sub("&nbsp;", " ", body)
    body = re.sub(ur"^[deleted]$", "", body) # Remove deleted
    body = re.sub("http[[:alnum:][:punct:]]*", " ", body) # Remove URL
    body = re.sub("/r/[[:alnum:]]+|/u/[[:alnum:]]+", " ", body) # Remove /r/subreddit, /u/user
    # body = re.sub("(>.*?\\n\\n)+", " ", body) # Remove quoted comments
    body = re.sub("[[:cntrl:]]", " ", body) # Remove control characters (\n, \b)
    body = re.sub("'", "", body) # Remove single quotation marks (contractions)
    body = re.sub("[[:punct:]]", " ", body) # Remove punctuation
    body = re.sub("\\s+", " ", body) # Replace multiple spaces with single space
    body = body.strip() # doesn't work for unicode
    # body = body.decode('utf-8').strip()
    body = body.lower() # Lower case
    return body # Return body (cleaned up text)

In [ ]:
def label(score):
    if int(score) <= -1: return 'neg'
    else: return 'pos'

In [ ]:
# clean up body, change numerical score to pos or neg
pflist_clean = map(lambda line: [cleanup(line[0]), label(line[1])], pflist_body_score)

In [ ]:
pflist_clean[100:150]

In [ ]:
len(pflist_clean)

In [ ]:
pflist_clean[1]

In [ ]:
# pflist_unicode = map(lambda line: [unicode(line[0]), line[1]], pflist_clean)

In [ ]:
pflist_tokens = map(lambda line: [nltk.word_tokenize(line[0]), line[1]], pflist_clean)

In [ ]:
pflist_clean

SEE ABOVE.  u'[deleted]' is still there.  cleanup() isn't working.  I need to set up test to debug and make sure it workds- TDD!!!  The book "Python Cookbook" might have the answers for doing regex with Unicode, but check free sources first.

In [ ]:
with open('data/test.csv', 'rb') as f:
    # reader = UnicodeReader(f)
    reader = unicodecsv.reader(f, encoding='utf-8')
    pflist = list(reader)